In [1]:
!pip install transformers

In [2]:
1
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [28]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

maximum_length = 16
caption_len = {"max_length": maximum_length}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values

  output_ids = model.generate(pixel_values, **caption_len)

  caption = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  return caption


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [29]:
predict_step(['/content/1003163366_44323f5815.jpg'])

['a man laying on a bench with a dog ']

In [30]:
predict_step(['/content/96399948_b86c61bfe6.jpg'])

['a man standing next to a large white object ']

In [27]:
import pickle

In [6]:
with open('image_model5.pkl', 'wb') as f:
    pickle.dump({
        'model': model,
        'feature_extractor': feature_extractor,
        'tokenizer': tokenizer,
        'gen_kwargs': caption_len
    }, f)


In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds
